In [ ]:
import os
import numpy as np
import random
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
tfk = tf.keras
print(tf.__version__)
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import regularizers

In [ ]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
dataset_dir = '../input/leavestrainin/training'

In [ ]:
image_generator_train = tfk.preprocessing.image.ImageDataGenerator(rescale = 1/255 , 
                                                             rotation_range = 75,
                                                             height_shift_range = 50,
                                                             width_shift_range= 50,
                                                             zoom_range = 0.3,
                                                             horizontal_flip = True,
                                                             vertical_flip=True,
                                                             fill_mode='reflect',
                                                             validation_split=0.1)

train_data = image_generator_train.flow_from_directory(dataset_dir,
                                                 target_size=(256,256),
                                                 color_mode = 'rgb',
                                                 classes=None,
                                                 batch_size =64,
                                                 shuffle = True,
                                                 seed = seed,
                                                 subset = "training")


#For validation is necessary to create an ImageDataGenerator without data augmentation
image_generator_validation = tfk.preprocessing.image.ImageDataGenerator(rescale = 1/255 , 
                                                             validation_split=0.1)


validation_data = image_generator_validation.flow_from_directory(dataset_dir,
                                                 target_size=(256,256),
                                                 color_mode = 'rgb',
                                                 classes=None,
                                                 batch_size =64,
                                                 shuffle = False,
                                                 seed = seed,
                                                 subset = "validation")

In [ ]:
input_shape = (256,256,3)

In [ ]:
supernet = tfk.applications.VGG16(
    include_top = False,
    weights = 'imagenet',
    input_shape = input_shape
)

supernet.summary()

In [ ]:
#put false to do transfer elarning, put true to do fine tuning
supernet.trainable = False

def create_model(input_shape):
  model = tfk.Sequential()
  model.add(supernet)
  model.add(tfk.layers.GlobalMaxPooling2D())
  model.add(tfk.layers.Dense(
      units = 512,
      activation = "relu",
      kernel_initializer=tfk.initializers.GlorotUniform(seed)
  ))
  model.add(tfk.layers.Dropout(0.5,seed=seed))
  #output layer
  model.add(tfk.layers.Dense(
      units = 14,
      activation = "softmax",
      kernel_initializer=tfk.initializers.GlorotUniform(seed)
  ))

  #compile model
  model.compile(loss=tfk.losses.CategoricalCrossentropy() ,
               optimizer = tfk.optimizers.Adam(),
               metrics='accuracy' )

  return model

In [ ]:
#callbacks (earlystopping)
callbacks = []
callbacks.append(tf.keras.callbacks.EarlyStopping(monitor ='val_loss', patience=5, restore_best_weights=True))

In [ ]:
#build the model
cnn_model = create_model(input_shape)
cnn_model.summary()

In [ ]:
#Train the model
history = cnn_model.fit(x = train_data,
                        validation_data= validation_data,
                        epochs = 100,
                        callbacks = callbacks ).history

In [ ]:
#save the model 
cnn_model.save("./CNN_TL_GAP_model")

In [ ]:
#zip the folder in order to be able to dowload it
!zip -r file3.zip "./CNN_TL_GAP_model"